In [1]:
%cd /home/elijah/video_membership


/home/elijah/video_membership


In [2]:
from app import db
from app.playlist.models import Playlist 
from app.users.models import User
from app.videos.models import Video
from cassandra.cqlengine.management import sync_table

In [3]:
from app.indexing.schemas import *

In [4]:
db.get_session()
sync_table(User)
sync_table(Video)
sync_table(Playlist)

In [5]:
import uuid 
import json
from pydantic import BaseModel , Field, validator, root_validator
from typing import Optional

In [6]:
video_obj = dict(Video.objects.first())
video_obj

{'host_id': 'RB-RcX5DS5A',
 'db_id': UUID('9ea79ce6-b3a7-11ed-a62a-d91575de2f44'),
 'host_service': 'youtube',
 'title': 'sammy ',
 'url': 'https://www.youtube.com/watch?v=RB-RcX5DS5A',
 'user_id': UUID('e2f76ba4-adc3-11ed-b906-331c8dd1903c')}

In [7]:
class VideoIndexSchema2(BaseModel):
    objectID: str = Field(alias='host_id')
    objectType: str = "Video"
    title: Optional[str]
    path: str = Field(alias='host_id')
        
    @validator("path")
    def set_path(cls, v, values, **kwargs):
        host_id = v
        return f"/videos/{host_id}"

In [8]:
VideoIndexSchema(**video_obj).dict()

{'objectID': 'RB-RcX5DS5A',
 'objectType': 'Video',
 'title': 'sammy ',
 'path': '/videos/RB-RcX5DS5A'}

In [9]:
playlist_obj = dict(Playlist.objects.first())
playlist_obj

{'db_id': UUID('101df0e6-b79a-11ed-ab0e-a9ea9ce579b1'),
 'user_id': None,
 'updated': datetime.datetime(2023, 3, 5, 5, 33, 19, 260000),
 'host_ids': ['0n-vUvMFffhH4'],
 'title': 'Hello sam'}

In [10]:
class PlaylistIndexSchema2(BaseModel):
    objectID: uuid.UUID = Field(alias='db_id')
    objectType: str = "Playlist"
    title: Optional[str]
    path: str = Field(default='/')
    
    @root_validator
    def set_default(cls, values):
        objectID = values.get('objectID')
        values['objectID'] = str(objectID)
        values['path'] = f"/playlists/{objectID}"
        return values

In [11]:
##json.loads(PlaylistIndexSchema(**playlist_obj).json())

In [12]:
PlaylistIndexSchema(**playlist_obj).dict()

{'objectID': '101df0e6-b79a-11ed-ab0e-a9ea9ce579b1',
 'objectType': 'Playlist',
 'title': 'Hello sam',
 'path': '/playlists/101df0e6-b79a-11ed-ab0e-a9ea9ce579b1'}

In [13]:
video_q = [dict(x) for x in Video.objects.all()]
videos_dataset = [VideoIndexSchema(**x).dict() for x in video_q]
videos_dataset

[{'objectID': 'RB-RcX5DS5A',
  'objectType': 'Video',
  'title': 'sammy ',
  'path': '/videos/RB-RcX5DS5A'},
 {'objectID': 'kNlcoQh2PnI',
  'objectType': 'Video',
  'title': 'Shark tank',
  'path': '/videos/kNlcoQh2PnI'},
 {'objectID': 'd-aEy9Ze01E',
  'objectType': 'Video',
  'title': 'sammy best viddeo33',
  'path': '/videos/d-aEy9Ze01E'},
 {'objectID': 'd-aEy9Ze01E',
  'objectType': 'Video',
  'title': 'THis is my new friend',
  'path': '/videos/d-aEy9Ze01E'},
 {'objectID': 'r5y4EDxiMWs',
  'objectType': 'Video',
  'title': None,
  'path': '/videos/r5y4EDxiMWs'},
 {'objectID': 'bjW4T3xp8vc',
  'objectType': 'Video',
  'title': 'Shark tank 2334 ',
  'path': '/videos/bjW4T3xp8vc'},
 {'objectID': '7cpWa0La_AA',
  'objectType': 'Video',
  'title': 'Jet bys',
  'path': '/videos/7cpWa0La_AA'},
 {'objectID': 'cjVQ36NhbMk',
  'objectType': 'Video',
  'title': None,
  'path': '/videos/cjVQ36NhbMk'},
 {'objectID': '2K2LpF1WdLk',
  'objectType': 'Video',
  'title': 'ANother video ',
  'path': 

In [14]:
playlist_q = [dict(x) for x in Playlist.objects.all()]
playlist_dataset = [PlaylistIndexSchema(**x).dict() for x in playlist_q]
playlist_dataset

[{'objectID': '101df0e6-b79a-11ed-ab0e-a9ea9ce579b1',
  'objectType': 'Playlist',
  'title': 'Hello sam',
  'path': '/playlists/101df0e6-b79a-11ed-ab0e-a9ea9ce579b1'},
 {'objectID': 'd0914642-b9df-11ed-b9de-6ff720f1ee13',
  'objectType': 'Playlist',
  'title': 'EDF',
  'path': '/playlists/d0914642-b9df-11ed-b9de-6ff720f1ee13'},
 {'objectID': '4a7d2062-b7fa-11ed-ab0e-a9ea9ce579b1',
  'objectType': 'Playlist',
  'title': 'sams best',
  'path': '/playlists/4a7d2062-b7fa-11ed-ab0e-a9ea9ce579b1'},
 {'objectID': 'fea5c0cc-b7f9-11ed-ab0e-a9ea9ce579b1',
  'objectType': 'Playlist',
  'title': 'video trailers',
  'path': '/playlists/fea5c0cc-b7f9-11ed-ab0e-a9ea9ce579b1'},
 {'objectID': '2a269742-b9dc-11ed-b9de-6ff720f1ee13',
  'objectType': 'Playlist',
  'title': 'sam',
  'path': '/playlists/2a269742-b9dc-11ed-b9de-6ff720f1ee13'},
 {'objectID': '93fd9398-b9df-11ed-b9de-6ff720f1ee13',
  'objectType': 'Playlist',
  'title': 'erf',
  'path': '/playlists/93fd9398-b9df-11ed-b9de-6ff720f1ee13'},
 {'ob

In [15]:
dataset = playlist_dataset + videos_dataset

In [17]:
dataset

[{'objectID': '101df0e6-b79a-11ed-ab0e-a9ea9ce579b1',
  'objectType': 'Playlist',
  'title': 'Hello sam',
  'path': '/playlists/101df0e6-b79a-11ed-ab0e-a9ea9ce579b1'},
 {'objectID': 'd0914642-b9df-11ed-b9de-6ff720f1ee13',
  'objectType': 'Playlist',
  'title': 'EDF',
  'path': '/playlists/d0914642-b9df-11ed-b9de-6ff720f1ee13'},
 {'objectID': '4a7d2062-b7fa-11ed-ab0e-a9ea9ce579b1',
  'objectType': 'Playlist',
  'title': 'sams best',
  'path': '/playlists/4a7d2062-b7fa-11ed-ab0e-a9ea9ce579b1'},
 {'objectID': 'fea5c0cc-b7f9-11ed-ab0e-a9ea9ce579b1',
  'objectType': 'Playlist',
  'title': 'video trailers',
  'path': '/playlists/fea5c0cc-b7f9-11ed-ab0e-a9ea9ce579b1'},
 {'objectID': '2a269742-b9dc-11ed-b9de-6ff720f1ee13',
  'objectType': 'Playlist',
  'title': 'sam',
  'path': '/playlists/2a269742-b9dc-11ed-b9de-6ff720f1ee13'},
 {'objectID': '93fd9398-b9df-11ed-b9de-6ff720f1ee13',
  'objectType': 'Playlist',
  'title': 'erf',
  'path': '/playlists/93fd9398-b9df-11ed-b9de-6ff720f1ee13'},
 {'ob